In [1]:
import sys
import urllib3
from bs4 import BeautifulSoup

sys.path.append('/Users/suzukiharumasaru/apps/WG-RPA/pytools')

import pandas as pd
from gsheets import GSheets
# インスタンスを生成
creds = '/Users/suzukiharumasaru/apps/WG-RPA/seacrets/rpa-shokikikaku-a6bba716cf27.json'
gs = GSheets(creds)

# 必要な情報を取得する
input_id = '1hQikhwgt8VSNC6GcfhFhHqQPAfoLygBJrK0cG_Ja3xc'
first_prep_id = '1Dt6KNrC98EjJ3hUxaKwguWfXvWP32QPspTcHrvUyldY'

input_sheets = gs.gc.open_by_key(input_id)
first_prep_sheets = gs.gc.open_by_key(first_prep_id)

In [2]:
input_sheet = input_sheets.worksheet('input')
content = input_sheet.get_values()
#シートの中身をデータフレーム化
input_df = pd.DataFrame(content, columns = content[0]).drop(0)
service_comps = input_df["カテゴリ系競合（最大8つまで）※除外するディレクトリがある場合には『,』を記載してください"][1].splitlines()
article_comps = input_df["記事系競合（最大8つまで）"][1].splitlines()

In [3]:
service_comps

['https://www.wifi-rental.com/,media',
 'https://www.netage.ne.jp/,column',
 'https://wifistore.jp/,column',
 'https://www.rental-store.jp/',
 'https://wifi-rent.jp/']

In [12]:
dicts={}
http = urllib3.PoolManager()
for i, comp in enumerate(service_comps):
    key = service_comps[i].split(",")[0]
    response = http.request('get', key)
    soup = BeautifulSoup(response.data, 'html.parser')
    title = soup.find('title').text
    try:
        val = service_comps[i].split(",")[1]
    except:
        val = ''
    dicts[key] = [val,title]
dicts

{'https://www.wifi-rental.com/': ['media', '【公式】WiFiレンタルどっとこむ｜WiFiレンタル日本国内専用'],
 'https://www.netage.ne.jp/': ['column',
  '【公式】NETAGE 日本国内 WiFi レンタル 出張・旅行・帰国などにとっても便利！'],
 'https://wifistore.jp/': ['column', '【公式】WiFiストア | 各社WiFiを徹底比較'],
 'https://www.rental-store.jp/': ['', '【WiFiレンタル屋さん】\u3000日本国内用WiFiルーターレンタル店'],
 'https://wifi-rent.jp/': ['', '国内用wifiレンタル 縁人 | Wi-Fiレンタルサービス | ']}

In [17]:
for i,items in enumerate(dicts.items()):
    print(items[1][1])

【公式】WiFiレンタルどっとこむ｜WiFiレンタル日本国内専用
【公式】NETAGE 日本国内 WiFi レンタル 出張・旅行・帰国などにとっても便利！
【公式】WiFiストア | 各社WiFiを徹底比較
【WiFiレンタル屋さん】　日本国内用WiFiルーターレンタル店
国内用wifiレンタル 縁人 | Wi-Fiレンタルサービス | 


In [18]:
first_prep_sheet=first_prep_sheets.worksheet('【#16】競合キーワード抽出「初期設定」')
ds= first_prep_sheet.range('D11:D18')
for i,items in enumerate(dicts.items()):
    ds[i].value = items[0]

first_prep_sheet.update_cells(ds)

ds= first_prep_sheet.range('E11:E18')
for i,items in enumerate(dicts.items()):
    ds[i].value = items[1][0]

ds= first_prep_sheet.range('C11:C18')
for i,items in enumerate(dicts.items()):
    ds[i].value = items[1][1]

first_prep_sheet.update_cells(ds)

{'spreadsheetId': '1Dt6KNrC98EjJ3hUxaKwguWfXvWP32QPspTcHrvUyldY',
 'updatedRange': "'【#16】競合キーワード抽出「初期設定」'!C11:C18",
 'updatedRows': 8,
 'updatedColumns': 1,
 'updatedCells': 8}

In [24]:
article_dicts={}
http = urllib3.PoolManager()
for comp in article_comps:
    key = comp
    print(key)
    response = http.request('get', key)
    soup = BeautifulSoup(response.data, 'html.parser')
    title = soup.find('title').text
    
    article_dicts[key] = title

article_dicts

https://www.wifi-rental.com/media/
https://wimax-broad.jp/column/
https://www.just-size.net/internet/
https://shibarinashi-wifi.jp/media/
https://www.mobistar.jp/
https://wimax比較.com/


{'https://www.wifi-rental.com/media/': 'TRENDi  生活に役立つトレンド・デジタル情報をお届け',
 'https://wimax-broad.jp/column/': 'Broad WiMAX 通信 | WiMAXのおすすめ比較/最新情報を発信する情報サイト',
 'https://www.just-size.net/internet/': 'ネット比較・検証｜Wi-Fiの森 | インターネット回線の専門家が監修する総合情報メディア',
 'https://shibarinashi-wifi.jp/media/': 'コムナビ',
 'https://www.mobistar.jp/': '海外携帯レンタル比較ナビ｜海外・国内用Wi-Fi携帯の比較情報サイト',
 'https://wimax比較.com/': 'WiMAX比較.com~おすすめプロバイダ23社ランキング2023年2月'}

In [32]:
first_prep_sheet=first_prep_sheets.worksheet('【#16】競合キーワード抽出「初期設定」')
ds= first_prep_sheet.range('J11:J18')
for i,items in enumerate(article_dicts.items()):
    ds[i].value = items[0]

first_prep_sheet.update_cells(ds)

ds= first_prep_sheet.range('I11:I18')
for i,items in enumerate(article_dicts.items()):
    ds[i].value = items[1]

first_prep_sheet.update_cells(ds)

{'spreadsheetId': '1Dt6KNrC98EjJ3hUxaKwguWfXvWP32QPspTcHrvUyldY',
 'updatedRange': "'【#16】競合キーワード抽出「初期設定」'!I11:I18",
 'updatedRows': 8,
 'updatedColumns': 1,
 'updatedCells': 8}

In [31]:
for i,items in enumerate(article_dicts.items()):
    print(items[0])

https://www.wifi-rental.com/media/
https://wimax-broad.jp/column/
https://www.just-size.net/internet/
https://shibarinashi-wifi.jp/media/
https://www.mobistar.jp/
https://wimax比較.com/


In [28]:
article_dicts

{'https://www.wifi-rental.com/media/': 'TRENDi  生活に役立つトレンド・デジタル情報をお届け',
 'https://wimax-broad.jp/column/': 'Broad WiMAX 通信 | WiMAXのおすすめ比較/最新情報を発信する情報サイト',
 'https://www.just-size.net/internet/': 'ネット比較・検証｜Wi-Fiの森 | インターネット回線の専門家が監修する総合情報メディア',
 'https://shibarinashi-wifi.jp/media/': 'コムナビ',
 'https://www.mobistar.jp/': '海外携帯レンタル比較ナビ｜海外・国内用Wi-Fi携帯の比較情報サイト',
 'https://wimax比較.com/': 'WiMAX比較.com~おすすめプロバイダ23社ランキング2023年2月'}